# EV長時間放置クラスタ予測ノートブック
要件.md（§0〜§8）に沿ってデータ前処理・学習・評価・成果物出力をhashvin単位で実行します。

## 0. セットアップ
ここではPythonパッケージと社内モジュールを読み込み、要件§0〜§2で求められる『hashvin単位の独立処理』を行う準備をします。

In [1]:
# §0〜§2: パス設定とユーティリティ読込（hashvinごとの独立処理を守る前提）
from pathlib import Path
import sys
import json
import pandas as pd

# ノートブックは multiclass_train ディレクトリで実行する想定（他hashvinの情報を混ぜない）
PROJECT_DIR = Path.cwd()
if str(PROJECT_DIR.parent) not in sys.path:
    sys.path.append(str(PROJECT_DIR.parent))

from multiclass_train.pipeline import FeatureToggleConfig, PipelineConfig, HashvinProcessor
from multiclass_train.trainer import train_and_evaluate

## 1. 設定
要件§3〜§7で利用するハイパーパラメータ・閾値を定義します。ステーション種別は将来的に有効化できるようフラグで制御します。

In [2]:
# §3〜§7: パイプライン設定（HEAD抽出K、特徴量トグルなど）
feature_toggles = FeatureToggleConfig(
    use_distance=True,          # MVP: 距離
    use_frequency=True,         # MVP: 頻度
    use_recency=True,           # MVP: Recency
    use_time_compat=True,       # MVP: time_compat
    use_behavior_flags=True,    # ＋α: 行動帯フラグ（必要に応じてOFFに切替）
    use_station_type=False,     # ＋α: ステーション種別（外部情報追加時にON推奨）
)

pipeline_config = PipelineConfig(
    head_k=10,  # §3: HEADクラスタの上限 K≤10
    feature_toggles=feature_toggles,
)

autogluon_presets = 'medium_quality'  # §6: AutoGluon Tabular のプリセット
autogluon_time_limit = 300  # 学習時間の上限（秒）
eval_thresholds = (0.5, 0.7, 0.9)  # §7: Top-1@τ の閾値候補

## 2. データ読込
要件§1の入力仕様に沿ってCSVを読み込み、処理対象のhashvinを確認します。セルの出力にはhashvin一覧が表示されます。

In [3]:
# §1: 充電イベントと放置イベントの入力CSVを読み込む
SESSIONS_PATH = PROJECT_DIR / 'ev_sessions_1.csv'
CHARGE_PATH = PROJECT_DIR / 'ev_charge_to_long_inactive.csv'

charge_df = pd.read_csv(CHARGE_PATH)
sessions_df = pd.read_csv(SESSIONS_PATH)

# hashvinごとの学習を徹底するため一覧を確認（出力: hashvinのリスト）
hashvin_list = sorted(charge_df['hashvin'].unique())
target_hashvins = hashvin_list  # 一部だけ試す場合はこのリストを編集
print(f'hashvin count: {len(hashvin_list)}')
hashvin_list

hashvin count: 1


['hv_0001_demo']

## 3. 学習・評価実行
要件§2〜§8をhashvin単位で順次実行し、
esult/<hashvin>/ に評価レポートや予測詳細を保存します。セルの標準出力には進捗とサマリ指標が表示されます。

In [4]:
# §2〜§8: hashvinごとの前処理・学習・評価・成果物保存フロー
RESULT_ROOT = PROJECT_DIR / 'result'
RESULT_ROOT.mkdir(parents=True, exist_ok=True)

metrics_summary = []
for hashvin in target_hashvins:
    print(f'=== Processing {hashvin} ===')  # 進捗ログ
    charge_subset = charge_df[charge_df['hashvin'] == hashvin].copy()
    sessions_subset = sessions_df[sessions_df['hashvin'] == hashvin].copy()

    processor = HashvinProcessor(hashvin, charge_subset, sessions_subset, pipeline_config)
    result = processor.build_features()  # §2〜§5: 特徴生成とHEAD抽出

    hashvin_dir = RESULT_ROOT / hashvin
    metrics = train_and_evaluate(
        result,
        hashvin_dir,
        enable_station_type=pipeline_config.enable_station_type,
        autogluon_presets=autogluon_presets,
        autogluon_time_limit=autogluon_time_limit,
        eval_thresholds=eval_thresholds,
    )  # §6〜§8: AutoGluon学習と評価レポート出力
    metrics_summary.append(metrics)

if metrics_summary:
    summary_path_json = RESULT_ROOT / 'metrics_summary.json'
    summary_path_csv = RESULT_ROOT / 'metrics_summary.csv'
    summary_path_json.write_text(json.dumps(metrics_summary, indent=2, ensure_ascii=False), encoding='utf-8')
    pd.DataFrame(metrics_summary).to_csv(summary_path_csv, index=False)
    display(pd.DataFrame(metrics_summary))  # Jupyter上で指標を可視化
else:
    print('No hashvin processed. Check target_hashvins list.')

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Memory Avail:       17.08 GB / 31.17 GB (54.8%)
Disk Space Avail:   835.16 GB / 930.73 GB (89.7%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'


=== Processing hv_0001_demo ===


Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "c:\workspace\src\kaggle\ml-study\EV-Battery-Parking-Degradation-Mitigation\multiclass_train\result\hv_0001_demo\autogluon"
Train Data Rows:    16
Train Data Columns: 48
Tuning Data Rows:    2
Tuning Data Columns: 48
Label Column:       y_class
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['I_202', 'I_101']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = I_202, class 0 = I_101
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (I_202) vs negative (I_101) class.
	To explicitly set the positive_class, either rename 

KeyError: "['charge_end_time'] not in index"